In [299]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from numpy import mean
from sklearn.utils import resample

# Read the data

In [300]:
X_full = pd.read_csv('/Users/macbookpro/Documents/2. ESCP/1. Classes/16. Machine Learning in Python/Assignment 2 & 3/train.csv')
X_full.head(10)

,label,id,visitTime,purchaseTime,hour,C1,C2,C3,C4,C5,...,N4,N5,N6,N7,N8,N9,N10,C10,C11,C12
0,-1,37,19274330,-1,0,2569487005,1830830742,4228834423,652481754,2967512603,...,0,0,0,0,0,0,0,419316267,3617851689,4220791646
1,-1,55,19274334,-1,0,3191875643,403381353,1238330876,652481754,2967512603,...,0,0,0,0,1,0,0,1968545380,3831440054,1213938795
2,-1,114,19274353,-1,0,2634604934,403381353,1928325921,652481754,2967512603,...,0,0,0,0,0,0,0,1405802835,2379633508,1284343215
3,-1,128,19274358,-1,0,2569487005,403381353,296915027,652481754,2967512603,...,0,0,0,0,0,0,0,2299319102,3617851689,4220791646
4,-1,164,19274367,-1,0,2569487005,403381353,681762206,652481754,2967512603,...,0,0,0,0,0,0,0,1306075469,3617851689,4220791646
5,-1,187,19274374,-1,0,3302106226,403381353,1928325921,652481754,2967512603,...,0,0,0,0,0,0,0,3182036626,2379633508,1284343215
6,-1,191,19274375,-1,0,4176412662,1830830742,2406323306,2308170622,2967512603,...,0,0,0,0,0,0,0,2514494748,3831440054,1213938795
7,-1,194,19274376,-1,0,3199195787,403381353,450016631,652481754,2967512603,...,0,0,0,0,0,0,0,655384408,621902171,1213938795
8,-1,212,19274384,-1,0,2569487005,403381353,3375570753,652481754,2967512603,...,0,0,0,0,0,0,0,1380655399,3617851689,4220791646
9,-1,227,19274387,-1,0,488865880,403381353,1129279372,652481754,2967512603,...,2,0,0,0,2,0,0,2984360656,3831440054,830706566


The dataset consists of a portion of our one-day traffic. The first column is the label: 1  indicates a purchase, -1 otherwise. The second column is the unique ID of the user session. The third and forth columns indicate the timestamp (in seconds) of the visit activity and the purchase one, respectively.

There are 10 features taking integer values (mostly count features, denoted by the prefix N) and 12 categorical features (denoted by the prefix C. Nb: they were not consecutively ordered). The values of the categorical features have been HASHED onto 32 bits for anonymization purposes. The semantic of these features is undisclosed. Some features may have missing values.

# Feature Selection

In [301]:
#Correlation Analysis
X_full.corr()[1:2]

,label,id,visitTime,purchaseTime,hour,C1,C2,C3,C4,C5,...,N4,N5,N6,N7,N8,N9,N10,C10,C11,C12
id,0.00651,1.0,0.985723,0.006543,0.984233,0.011131,-0.031077,-0.014507,0.048138,-0.044974,...,0.026743,0.00686,0.012628,-0.001135,0.035148,0.01339,0.012329,-0.00043,0.053341,0.070517


In [302]:
# select label as prediction target
y = X_full.label

# select features for prediction, droping the variables with high correlation and prediction target
features = ["N1","N2","N3","N4","N5","N6","N7","N8","N9","N10", 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12']
X = X_full[features]

#Validation and Train Sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.75, test_size=0.25, 
                                                      random_state=1, stratify = y)


# Managing unbalanced prediction target

In [303]:
n_purch = np.sum(X_full['label'] == 1)
n_non_purch = np.sum(X_full['label'] == -1)
print(f"Number of purchases in full data set: {n_purch}")
print(f"Number of non purchases in full data set: {n_non_purch}")

Number of purchases in full data set: 57
Number of non purchases in full data set: 31371


In [304]:
# concatenate our training data back together
X2 = pd.concat([X_train, y_train], axis=1)

# separate minority and majority classes
purchase = X2[X2.label==1]
not_purchase = X2[X2.label==-1]

# upsample minority
purchase_upsampled = resample(purchase,
                              replace=True, # sample with replacement
                              n_samples=len(not_purchase), # match number in majority class
                              random_state=1) # reproducible results

# combine majority and upsampled minority
upsampled = pd.concat([not_purchase, purchase_upsampled])

# check new class counts
upsampled.label.value_counts()

#redefine train data set
y_train = upsampled.label
X_train = upsampled[features]

# Training the Logistic Regression Model

In [305]:
# trying logistic regression with the balanced dataset
lr_model = LogisticRegression()
lr_model.fit(X_train, y_train)
lr_y_pred = lr_model.predict(X_valid)

#Classification report
print("Classification Report")
print(classification_report(y_valid, lr_y_pred))

#Confusion Matrix
print("Confusion Matrix")
print(confusion_matrix(y_valid, lr_y_pred))

scores = cross_val_score(lr_model, X, y, scoring='roc_auc', n_jobs=-1)
print('Mean ROC AUC: %.3f' % mean(scores))

Classification Report
              precision    recall  f1-score   support

          -1       1.00      0.73      0.84      7843
           1       0.00      0.57      0.01        14

    accuracy                           0.73      7857
   macro avg       0.50      0.65      0.42      7857
weighted avg       1.00      0.73      0.84      7857

Confusion Matrix
[[5708 2135]
 [   6    8]]
Mean ROC AUC: 0.727


# Deploying model

In [306]:
#preparint test data set
test_data = pd.read_csv('/Users/macbookpro/Documents/2. ESCP/1. Classes/16. Machine Learning in Python/Assignment 2 & 3/test1.csv')
del test_data['label']
ID = test_data['id']
test_data = test_data.drop(['id', 'purchaseTime', 'visitTime','hour'], axis=1)
test_data.tail()

,C1,C2,C3,C4,C5,C6,C7,C8,N1,C9,...,N4,N5,N6,N7,N8,N9,N10,C10,C11,C12
32081,2569487005,2018710401,887116886,652481754,2967512603,3624739744,2303407174,157533148,0,4287642944,...,0,0,0,0,0,0,0,1546561973,3617851689,4220791646
32082,2569487005,403381353,3475347077,652481754,2967512603,4013717548,2303407174,3588031319,0,564075893,...,0,0,0,0,0,0,0,3227742820,3617851689,4220791646
32083,2569487005,403381353,3475347077,652481754,2967512603,4013717548,2303407174,3588031319,0,564075893,...,0,0,0,0,0,0,0,4069315994,3617851689,4220791646
32084,2569487005,2541203883,448895441,2308170622,2967512603,3624739744,2303407174,2206061889,0,4287642944,...,0,0,0,0,0,0,0,2724771230,3617851689,4220791646
32085,2569487005,403381353,2294620541,652481754,2967512603,3624739744,2303407174,157533148,0,4287642944,...,0,0,0,0,0,0,0,2151556883,3617851689,4220791646


In [307]:
#generate predictions with test data set
predictions = pd.DataFrame(lr_model.predict_proba(test_data), columns=['Prob_0','Prob_1'])
test_data['Prob_0'] = predictions['Prob_0']
test_data['Prob_1'] = predictions['Prob_1']
test_data['ID'] = ID

In [308]:
# generate results data frame
results = pd.DataFrame()
results['ID'] = test_data['ID']
results['Prob_0'] = test_data['Prob_0']
results['Prob_1'] = test_data['Prob_1']
results.set_index('ID')
results.to_csv("results.csv")